In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

%matplotlib inline
# 시각화 결과가 선명하게 표시되도록
%config InlineBackend.figure_format = 'retina'

df = pd.read_csv("word2vec_wrangling.csv")
df.shape

(61, 2)

In [2]:
df.head()

,exercise_name,Content_txt
0,PT,"💯 What I try to educate my clients around, doe..."
1,검도,#20200115\n저녁 초대!\n와인잔 속에 비치는\n모든 것들이 화려한\n도심속...
2,기구필라테스,#오늘의동작\n캐딜락 동작의 완성 '행잉'\n⠀\n중력을 이용해 척추를 늘려주고\n...
3,다빈치바디보드,#mbn생생정보마당 \n#고투\n#고투GX\n#다빈치바디보드\n#생방송 #GOTOL...
4,드럼스틱,#드럼스틱 #고무팁 #테크라스틱 #전자드럼용스틱\n\n1. 전자드럼타격시 덜 시끄럽...


In [3]:
import soynlp
import gensim

In [4]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [5]:
def open_file(exercise_name):
    file_name = "#" + exercise_name + "_sum.txt"
    file_1 = "/Users/noopy/FitCuration/" + file_name
    text = open(file_1, 'r', -1, "UTF-8", errors="ignore").read()
    return text

In [6]:
sample_content = open_file("자이로토닉")

In [7]:
sample_content = preprocessing(sample_content)
sample_content

'   자이로토닉  gyrotonic 비교할 수 없는 레슨 퀄리티     월 프롬과 함께하세요   위례신도시 No      월 이벤트  신규 개인레슨   회   만원 선착순   분   신규 그룹레슨 최대    할인 개월수 별로 할인율 상이    인 이상 단체등록 시 신규클래스 오픈      인 소수 그룹레슨 미국 BBU 장비        개인레슨 미국 Gratz  BBU  Gyrotonic 장비     클래식필라테스  자이로토닉    프롬필라테스 위례동로     우성메디피아  층                  안녕하세요  에코필라테스 플러스입니다     설 연휴 전까지 열심히 운동하러 오시는 우리 멋찐 회원님들을 맞이하려 저는 출근하자마자 부랴부랴 청소를 끝마쳤답니다      어서 센터를 뜨겁게 달구어 주세요     오늘도 평화로운 에코필라테스 플러스와 함께        자세한 문의 및 상담                 카카오 오픈채팅   에코필라테스플러스   blog naver com ecopilatesplus   첨먹어보는 춰컬릿    사만다 에듀케이터  samantha jrobinsonhk 의 달달한간식   맛있다   ㅎㅎ자꾸먹게되는군요      아치앤컬  필라테스  폴스타필라테스  자이로토닉  자이로토닉   자이로키네시스  자이로키네시스   청담필라테스  청담자이로토닉  부산필라테스  부산자이로토닉  archncurl  pilates  polestarpilates  gyrotonic  gyrokinesis 효지원장님  misa limepilates 의 선물    교육듣느라 안그래도 정신없을텐데 요렇게 또 챙겨주니 넘 고마워요 이쁘게 잘쓸게용         아치앤컬  필라테스  폴스타필라테스  자이로토닉  자이로토닉강사  자이로키네시스  자이로키네시스   청담필라테스  부산필라테스  청담자이로토닉  부산자이로토닉  archncurl  pilates  polestarpilates  gyrotonic  gyrokinesis  지유필라테스 자이로토닉 G YOU

In [9]:
# word2vec 모델 학습에 로그를 찍을 수 있도록 합니다.
import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)

In [10]:
# %time을 찍어주면 해당 코드를 실행할 때 걸리는 시간을 출력해 줍니다
%time sentences = df['Content_txt'].apply(preprocessing)

CPU times: user 754 ms, sys: 6.97 ms, total: 761 ms
Wall time: 760 ms


In [13]:
from soynlp.tokenizer import RegexTokenizer

tokenizer = RegexTokenizer()
tokenizer

/Users/noopy/opt/anaconda3/lib/python3.7/site-packages/soynlp/tokenizer/_tokenizer.py:19: FutureWarning: Possible nested set at position 13
  ('english & latin', re.compile(u"[a-zA-ZÀ-ÿ]+[[`']?s]*|[a-zA-ZÀ-ÿ]+", re.UNICODE))


In [14]:
# 전처리 이후의 샘플 텍스트로 토큰화
tokened_content = tokenizer.tokenize(sample_content)
tokened_content[:10]

['자이로토닉', 'gyrotonic', '비교할', '수', '없는', '레슨', '퀄리티', '월', '프롬과', '함께하세요']

In [15]:
print(len(tokened_content))

43663


In [22]:
# 초기화 및 모델 학습
from gensim.models import word2vec

%time tokens = sentences.apply(tokenizer.tokenize)

# 모델 학습
model = word2vec.Word2Vec(tokens, min_count=1)

model

2020-02-03 00:32:23,593 : INFO : collecting all words and their counts
2020-02-03 00:32:23,594 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


CPU times: user 16.6 s, sys: 23.9 ms, total: 16.6 s
Wall time: 16.6 s


2020-02-03 00:32:24,153 : INFO : collected 324341 word types from a corpus of 2300597 raw words and 61 sentences
2020-02-03 00:32:24,154 : INFO : Loading a fresh vocabulary
2020-02-03 00:32:25,685 : INFO : effective_min_count=1 retains 324341 unique words (100% of original 324341, drops 0)
2020-02-03 00:32:25,686 : INFO : effective_min_count=1 leaves 2300597 word corpus (100% of original 2300597, drops 0)
2020-02-03 00:32:26,489 : INFO : deleting the raw counts dictionary of 324341 items
2020-02-03 00:32:26,494 : INFO : sample=0.001 downsamples 9 most-common words
2020-02-03 00:32:26,495 : INFO : downsampling leaves estimated 2287695 word corpus (99.4% of prior 2300597)
2020-02-03 00:32:27,172 : INFO : estimated required memory for 324341 words and 100 dimensions: 421643300 bytes
2020-02-03 00:32:27,173 : INFO : resetting layer weights
2020-02-03 00:33:14,694 : INFO : training model with 3 workers on 324341 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2

In [23]:
stopwords_Twitter = ["입니다","있는","있습니다","같은","안녕하세요","고마워요","있어요","있게"\
                     ,"있도록","부탁드립니다","하는","합니다","할","하세요","하기","해","됩니다","하여",'잘','된','되고','되어','되었습니다',"없는","드립니다"\
                    ,"되기","하시는","하고","않을","같다","싶다","이런","저런","그런",'바랍니다'\
                    ,"했습니다","했다","해드립니다","하신","하실","않고","해요","가능합니다","하고싶으신"\
                    ,"않으며","주세요","오세요"]

stopwords_mecab = ['수','퀄리티','도시','분','전문','스타','년','원',\
                   '월','화','수','목','금','시','앤','일','그램','문'] 

stopwords = stopwords_mecab + stopwords_Twitter

In [24]:
# 모델 이름을 지정하고 저장한다.
model_name = 'my_second_model'
model.save(model_name)

2020-02-03 00:33:25,025 : INFO : saving Word2Vec object under my_second_model, separately None
2020-02-03 00:33:25,026 : INFO : storing np array 'vectors' to my_second_model.wv.vectors.npy
2020-02-03 00:33:25,323 : INFO : not storing attribute vectors_norm
2020-02-03 00:33:25,324 : INFO : storing np array 'syn1neg' to my_second_model.trainables.syn1neg.npy
2020-02-03 00:33:25,607 : INFO : not storing attribute cum_table
2020-02-03 00:33:26,140 : INFO : saved my_second_model


In [25]:
# 단어 사전 수
len(model.wv.vocab)

324341

In [26]:
# 단어 사전에서 상위 10개만 보기
vocab = model.wv.vocab
sorted(vocab, key=vocab.get, reverse=True)[:30]

['운동',
 '다이어트',
 '요가',
 '필라테스',
 '월',
 '시',
 '일',
 '운동하는여자',
 '수',
 '일상',
 'a',
 '더',
 't',
 '분',
 '함께',
 '년',
 '문의',
 'pt',
 '있는',
 '운동하는남자',
 'PT',
 '헬스',
 '살사',
 '플라잉요가',
 '층',
 '너무',
 '바차타',
 'yoga',
 '있습니다',
 '시간']

In [27]:
# Counter로 자주 등장하는 단어 보기
from collections import Counter
dict(Counter(vocab).most_common(20))

{'운동': <gensim.models.keyedvectors.Vocab at 0x7fb3b0c0e550>,
 '다이어트': <gensim.models.keyedvectors.Vocab at 0x7fb3b0c0e610>,
 '요가': <gensim.models.keyedvectors.Vocab at 0x7fb3aac11b90>,
 '필라테스': <gensim.models.keyedvectors.Vocab at 0x7fb3f377d710>,
 '월': <gensim.models.keyedvectors.Vocab at 0x7fb3aac202d0>,
 '시': <gensim.models.keyedvectors.Vocab at 0x7fb3aac51690>,
 '일': <gensim.models.keyedvectors.Vocab at 0x7fb3aac568d0>,
 '운동하는여자': <gensim.models.keyedvectors.Vocab at 0x7fb3b0c0e5d0>,
 '수': <gensim.models.keyedvectors.Vocab at 0x7fb3aabd8050>,
 '일상': <gensim.models.keyedvectors.Vocab at 0x7fb3f377d810>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7fb3a9f69950>,
 '더': <gensim.models.keyedvectors.Vocab at 0x7fb43dd41bd0>,
 't': <gensim.models.keyedvectors.Vocab at 0x7fb3a9a73050>,
 '분': <gensim.models.keyedvectors.Vocab at 0x7fb41cce6710>,
 '함께': <gensim.models.keyedvectors.Vocab at 0x7fb3aac244d0>,
 '년': <gensim.models.keyedvectors.Vocab at 0x7fb3dff72b10>,
 '문의': <gensim.models.key

In [28]:
# 가장 적게 등장하는 단어
min(vocab, key=vocab.get)

'encourage'

In [34]:
model.wv.most_similar('자이로토닉')

2020-02-02 23:42:35,145 : INFO : precomputing L2-norms of word weight vectors


[('자이로키네시스', 0.9982904195785522),
 ('발레핏', 0.9974616765975952),
 ('가희', 0.9970693588256836),
 ('ttc', 0.996626079082489),
 ('다빈치바디보드', 0.9961905479431152),
 ('클래식필라테스', 0.9957948923110962),
 ('폼롤러', 0.995484471321106),
 ('여성전용', 0.9954193830490112),
 ('설명절', 0.9949073791503906),
 ('부산요가', 0.9943119883537292)]

In [35]:
model.wv.most_similar('복근')

[('복근운동', 0.9992300868034363),
 ('모델', 0.998993992805481),
 ('런', 0.9987736940383911),
 ('합기도', 0.9985851049423218),
 ('폴댄스', 0.9985740184783936),
 ('광주', 0.9984042048454285),
 ('상인동', 0.9982923269271851),
 ('코어운동', 0.9982035160064697),
 ('태권도', 0.9981626868247986),
 ('갑니다', 0.9981544017791748)]

In [36]:
model.wv.most_similar('척추')

[('여전히', 0.9987314343452454),
 ('우한', 0.9985310435295105),
 ('어느새', 0.9985133409500122),
 ('엄청', 0.9984723329544067),
 ('운동하며', 0.9984238743782043),
 ('울', 0.9984229207038879),
 ('함께해서', 0.9984023571014404),
 ('하다가', 0.9983784556388855),
 ('푹', 0.9983559250831604),
 ('평생', 0.9983251094818115)]

In [44]:
model.wv.most_similar(positive=['척추','건강'], negative=['우한'])

[('타바타운동', 0.9961912631988525),
 ('유연성', 0.9956024289131165),
 ('교정', 0.9955008029937744),
 ('근육학', 0.9954005479812622),
 ('벌크업', 0.9948692321777344),
 ('홈트', 0.9947450757026672),
 ('코어강화', 0.9945970773696899),
 ('근육기능운동', 0.9940768480300903),
 ('헬스장', 0.9939762949943542),
 ('운동', 0.9938791394233704)]

In [45]:
model.wv.most_similar(positive=['척추','건강'])

[('대한폴댄스연맹', 0.9983609318733215),
 ('여름', 0.9979842901229858),
 ('MMA', 0.99770587682724),
 ('점핑', 0.9976186156272888),
 ('가즈아', 0.9975834488868713),
 ('운동영상', 0.9973958730697632),
 ('우아하게', 0.9971697926521301),
 ('필라테스하는여자', 0.9971282482147217),
 ('수영장', 0.9970409870147705),
 ('샐러드', 0.9968857169151306)]

In [48]:
model.wv.similarity('척추', '검도')

0.9884146

In [29]:
exercise_to_loop = ["자이로토닉","필라테스", "요가","빈야사","키네시스","기구필라테스","매트필라테스","소도구필라테스","플라잉필라테스","헬스","핫요가","플라잉","하타요가","아쉬탕가","인요가","요가쿠아","번지요가","에이리얼후프","발레핏","수영","등산","조깅","크로스핏","사이클","스피닝",
"타바타","복싱","플라잉요가","파운드핏","드럼스틱","주짓수","번지피트니스","폴댄스","방송댄스","에어로빅","뮤직복싱","번지댄스","줌바","힙레","재즈댄스","살사","바차타","라틴댄스","탱고","펜싱","검도","암벽등반","필록싱","다빈치바디보드","점핑피트니스","트램폴린","볼링","수상스키","서핑","패들보드",\
                     "아쿠아바이크","패들핏","아쿠아로빅","아쿠아테크","PT","발레"]

exercise_to_loop.sort()
exercise_to_loop

['PT',
 '검도',
 '기구필라테스',
 '다빈치바디보드',
 '드럼스틱',
 '등산',
 '라틴댄스',
 '매트필라테스',
 '뮤직복싱',
 '바차타',
 '발레',
 '발레핏',
 '방송댄스',
 '번지댄스',
 '번지요가',
 '번지피트니스',
 '복싱',
 '볼링',
 '빈야사',
 '사이클',
 '살사',
 '서핑',
 '소도구필라테스',
 '수상스키',
 '수영',
 '스피닝',
 '아쉬탕가',
 '아쿠아로빅',
 '아쿠아바이크',
 '아쿠아테크',
 '암벽등반',
 '에어로빅',
 '에이리얼후프',
 '요가',
 '요가쿠아',
 '인요가',
 '자이로토닉',
 '재즈댄스',
 '점핑피트니스',
 '조깅',
 '주짓수',
 '줌바',
 '크로스핏',
 '키네시스',
 '타바타',
 '탱고',
 '트램폴린',
 '파운드핏',
 '패들보드',
 '패들핏',
 '펜싱',
 '폴댄스',
 '플라잉',
 '플라잉요가',
 '플라잉필라테스',
 '필라테스',
 '필록싱',
 '하타요가',
 '핫요가',
 '헬스',
 '힙레']

In [66]:
vector_val = []

def tuple(exercise_name,word):
    return (word, model.wv.similarity(exercise_name,word))

def show_similarity_for_each_exercise(word):
    for item in exercise_to_loop:
        vector_val.append(tuple(word, item))
    return vector_val

vector_list = show_similarity_for_each_exercise('재활')

In [67]:
def Sort(tup): 
    return(sorted(tup, key = lambda x: float(x[1]), reverse = True)) 

Sort(vector_list)

[('매트필라테스', 0.98873556),
 ('타바타', 0.9835689),
 ('소도구필라테스', 0.98279935),
 ('기구필라테스', 0.9801476),
 ('헬스', 0.9770445),
 ('빈야사', 0.9754158),
 ('핫요가', 0.97496533),
 ('플라잉요가', 0.9733168),
 ('필라테스', 0.97159535),
 ('요가', 0.96709335),
 ('줌바', 0.9643829),
 ('스피닝', 0.9613105),
 ('아쉬탕가', 0.96072805),
 ('하타요가', 0.95482606),
 ('에이리얼후프', 0.9535863),
 ('발레핏', 0.95178795),
 ('뮤직복싱', 0.9500154),
 ('필록싱', 0.9481107),
 ('패들핏', 0.9479432),
 ('PT', 0.9457139),
 ('파운드핏', 0.94387436),
 ('자이로토닉', 0.9406905),
 ('다빈치바디보드', 0.93899965),
 ('에어로빅', 0.93832743),
 ('크로스핏', 0.9381964),
 ('복싱', 0.9369781),
 ('번지피트니스', 0.93302953),
 ('수영', 0.92825806),
 ('발레', 0.9277791),
 ('플라잉필라테스', 0.92594445),
 ('인요가', 0.9198724),
 ('요가쿠아', 0.9176086),
 ('플라잉', 0.9159675),
 ('폴댄스', 0.91525286),
 ('번지댄스', 0.9009819),
 ('사이클', 0.9000862),
 ('번지요가', 0.8998159),
 ('힙레', 0.8728975),
 ('패들보드', 0.86603117),
 ('조깅', 0.86450005),
 ('키네시스', 0.8614933),
 ('주짓수', 0.8570968),
 ('암벽등반', 0.844186),
 ('점핑피트니스', 0.8404676),
 ('등산', 0.83684623),
 ('수